In [1]:
import copy
import pandas as pd
import plotly.express as px

# The Model

In [2]:
# Define parameters:

## Propogation parameters
represetations_decay_rate = 0.01          # The number of activation units lost every iteration (from each Stroop/LangKnow node)
cognitive_control_decay_rate = 0.005      # The number of activation units lost every iteration (from each CC node)
activation_rate = 0.1                     # The relation between unit activation and the activation it exerts through excitatory connections
monitor_biasing_activation_rate = 1       # The relation between unit activation and the activation it exerts through excitatory connections
inhibition_rate = 0.1                     # The relation between unit activation and the inhibition it exerts through inhibitory connections
biasing_mult = 0.00002                    # The relation between Biasing unit activation level and how strongly it activates Stroop/Lang high-level nodes (in proportion to their activation levels)

## Simulation parameters
max_iter = 5000                           # Maximum number of iterations (per trial)
activation_threshold = 1000               # Threshold activation level for a lower-level unit to be cosidered the final interpratation
between_trials_interval = 1.5             # Time (in seconds) between consecutive trials

## Input activations for each trial type (order: WK,MSK_ing,MSK_ed,text_blue,text_red,font_color)
congruent_stroop = (0,0,0,10,0,15)        # Activation of font_color that supports Blue & text_blue that supports Blue
incongruent_stroop = (0,0,0,0,10,15)      # Activation of font_color that supports Blue & text_red that supports red
congruent_sentence = (15,0,10,0,0,0)      # Activation of wk that supports SubjIsTheme & MSK_ed that supports SubjIsTheme
anomalous_sentence = (15,10,0,0,0,0)      # Activation of wk that supports SubjIsTheme & MSK_ing that supports SubjIsAgent


In [3]:
# Initialize
CogCtrl=None
LingKnow=None
Stroop=None

# Classes
class CognitiveControl(object):
    def __init__(self):
        self.conflict_monitoring = 0
        self.biasing = 0
 
    def update(self,LingKnow,Stoop):
        # increase CM activity based on the ratio between SubjIsAgent & SubjIsTheme
        if LingKnow.subj_is_agent+LingKnow.subj_is_theme > 0:
            self.conflict_monitoring += activation_rate*1/(abs(LingKnow.subj_is_agent-LingKnow.subj_is_theme))
        # increase CM activity based on the ratio between Blue & Red
        if Stroop.blue+Stroop.red > 0:
            self.conflict_monitoring += activation_rate*1/(abs(Stroop.blue-Stroop.red))
        # increase B activation based on CM activation
        self.biasing += monitor_biasing_activation_rate*self.conflict_monitoring       
        # decay
        self.conflict_monitoring -= cognitive_control_decay_rate
        self.biasing -= cognitive_control_decay_rate      
        # prevent negative activation values
        if self.conflict_monitoring < 0:
            self.conflict_monitoring = 0
        if self.biasing < 0:
            self.biasing = 0
    
    def between_trials_decay(self):
        self.conflict_monitoring -= cognitive_control_decay_rate*between_trials_interval*1000
        self.biasing -= cognitive_control_decay_rate*between_trials_interval*1000
        if self.conflict_monitoring < 0:
            self.conflict_monitoring = 0
        if self.biasing < 0:
            self.biasing = 0


            
class LinguisticKnowledge(object):
    def __init__(self, world_act=0, morphosynt_ing_act=0, morphosynt_ed_act=0):
        # Higher-Level
        self.world_knowledge = world_act
        self.morphosyntactic_knowledge_ing = morphosynt_ing_act
        self.morphosyntactic_knowledge_ed = morphosynt_ed_act
        # Lower-Level
        self.subj_is_agent = 0
        self.subj_is_theme = 0
        
    def input_activation(self,WK,MSK_ing,MSK_ed):
        self.world_knowledge += WK
        self.morphosyntactic_knowledge_ing += MSK_ing
        self.morphosyntactic_knowledge_ed += MSK_ed

    def update(self,CogCtrl,Stoop):        
        # save initial activations for calculations (to make sure all updating happens "at once")
        WorldKnowledgeInitialAct = self.world_knowledge
        MorphoSyntacticKnowledgeIngInitialAct = self.morphosyntactic_knowledge_ing
        MorphoSyntacticKnowledgeEdInitialAct = self.morphosyntactic_knowledge_ed
        SubjIsAgentInitialAct = self.subj_is_agent
        SubjIsThemeInitialAct = self.subj_is_theme

        # increase WK & MSK activity based on CogCtrl.biasing and their activation levels
        self.world_knowledge += (WorldKnowledgeInitialAct*CogCtrl.biasing*biasing_mult)
        self.morphosyntactic_knowledge_ing += (MorphoSyntacticKnowledgeIngInitialAct*CogCtrl.biasing*biasing_mult)        
        self.morphosyntactic_knowledge_ed += (MorphoSyntacticKnowledgeEdInitialAct*CogCtrl.biasing*biasing_mult)        
        
        # increase SubjIsAgent & SubjIsTheme based on WK & MSK
        self.subj_is_agent += (MorphoSyntacticKnowledgeIngInitialAct*activation_rate) # SubjIsAgent is supported by MSK_ing
        self.subj_is_theme += (MorphoSyntacticKnowledgeEdInitialAct*activation_rate) # SubjIsTheme is supported by MSK_ed
        self.subj_is_theme += (WorldKnowledgeInitialAct*activation_rate) # SubjIsTheme is supported by WN
        
        # lateral inhibition between SubjIsAgent & SubjIsTheme
        self.subj_is_agent -= (SubjIsThemeInitialAct*inhibition_rate)
        self.subj_is_theme -= (SubjIsAgentInitialAct*inhibition_rate)
        
        # decay
        self.world_knowledge -= represetations_decay_rate
        self.morphosyntactic_knowledge_ing -= represetations_decay_rate
        self.morphosyntactic_knowledge_ed -= represetations_decay_rate
        self.subj_is_agent -= represetations_decay_rate
        self.subj_is_theme -= represetations_decay_rate
        # prevent negative activation values
        if self.world_knowledge < 0 :
            self.world_knowledge = 0
        if self.morphosyntactic_knowledge_ing <0:
            self.morphosyntactic_knowledge_ing = 0
        if self.morphosyntactic_knowledge_ed <0:
            self.morphosyntactic_knowledge_ed = 0
        if self.subj_is_agent < 0:
            self.subj_is_agent = 0
        if self.subj_is_theme < 0:
            self.subj_is_theme = 0
    
    def between_trials_decay(self):
        self.world_knowledge -= represetations_decay_rate*between_trials_interval*1000
        self.morphosyntactic_knowledge_ing -= represetations_decay_rate*between_trials_interval*1000
        self.morphosyntactic_knowledge_ed -= represetations_decay_rate*between_trials_interval*1000
        self.subj_is_agent -= represetations_decay_rate*between_trials_interval*1000
        self.subj_is_theme -= represetations_decay_rate*between_trials_interval*1000
        if self.world_knowledge < 0 :
            self.world_knowledge = 0
        if self.morphosyntactic_knowledge_ing <0:
            self.morphosyntactic_knowledge_ing = 0
        if self.morphosyntactic_knowledge_ed <0:
            self.morphosyntactic_knowledge_ed = 0
        if self.subj_is_agent < 0:
            self.subj_is_agent = 0
        if self.subj_is_theme < 0:
            self.subj_is_theme = 0


        
class StroopTaskRepresentation(object):
    def __init__(self, text_blue_act=0, text_red_act=0, font_color_act=0):
        # Higher-Level
        self.text_blue = text_blue_act
        self.text_red = text_red_act
        self.font_color = font_color_act
        # Lower-Level
        self.blue = 0
        self.red = 0
            
    def input_activation(self,text_blue,text_red,font_color):
        self.text_blue += text_blue
        self.text_red += text_red
        self.font_color += font_color
        

    def update(self,CogCtrl,LingKnow):
        # save initial activations for calculations (to make sure all updating happens "at once")
        TextBlueInitialAct = self.text_blue
        TextRedInitialAct = self.text_red
        FontColorInitialAct = self.font_color
        BlueInitialAct = self.blue
        RedInitialAct = self.red
        
        # increase Text & FontColor activity based on CogCtrl.biasing and their activation levels
        self.text_blue += (TextBlueInitialAct*CogCtrl.biasing*biasing_mult)
        self.text_red += (TextRedInitialAct*CogCtrl.biasing*biasing_mult)
        self.font_color += (FontColorInitialAct*CogCtrl.biasing*biasing_mult)

        # increase Blue & Red based activity based on Text & FontColor
        self.blue += (FontColorInitialAct*activation_rate) # Blue is supported by FontColor
        self.blue += (TextBlueInitialAct*activation_rate) # Blue is supported by text_blue
        self.red += (TextRedInitialAct*activation_rate) # Red is supported by text_red

        # lateral inhibition between Blue & Red
        self.blue -= (RedInitialAct*inhibition_rate)
        self.red -= (BlueInitialAct*inhibition_rate)
        
        # decay
        self.text_blue -= represetations_decay_rate
        self.text_red -= represetations_decay_rate
        self.font_color -= represetations_decay_rate
        self.blue -= represetations_decay_rate
        self.red -= represetations_decay_rate
        # prevent negative activation values
        if self.text_blue < 0:
            self.text_blue = 0
        if self.text_red < 0:
            self.text_red = 0
        if self.font_color <0:
            self.font_color = 0
        if self.blue < 0:
            self.blue = 0
        if self.red < 0:
            self.red = 0

    def between_trials_decay(self):
        self.text_blue -= represetations_decay_rate*between_trials_interval*1000
        self.text_red -= represetations_decay_rate*between_trials_interval*1000
        self.font_color -= represetations_decay_rate*between_trials_interval*1000
        self.blue -= represetations_decay_rate*between_trials_interval*1000
        self.red -= represetations_decay_rate*between_trials_interval*1000
        if self.text_blue < 0:
            self.text_blue = 0
        if self.text_red < 0:
            self.text_red = 0
        if self.font_color <0:
            self.font_color = 0
        if self.blue < 0:
            self.blue = 0
        if self.red < 0:
            self.red = 0

        
# Functions
def update_all(CogCtrl,LingKnow,Stroop):    
    CogCtrl_prev = copy.copy(CogCtrl)
    LingKnow_prev = copy.copy(LingKnow)
    Stroop_prev = copy.copy(Stroop)
    
    CogCtrl.update(LingKnow_prev,Stroop_prev)
    LingKnow.update(CogCtrl_prev,Stroop_prev)
    Stroop.update(CogCtrl_prev,LingKnow_prev)
    return [CogCtrl, LingKnow, Stroop]


def run_trial(input_activation:tuple,CC=None,LK=None,S=None):
    '''
    input_activation specifies the input activations for WK,MSK_ing,MSK_ed,text_blue,text_red,font_color (in that order).
    '''
    i = 1
    activation_levels = {'ConfMon':[],'Biasing':[],
                        'WK':[],'MSK_ing':[],'MSK_ed':[],'SubjIsAgent':[],'SubjIsTheme':[],
                       'FontColor':[],'Text_blue':[],'Text_red':[],'Blue':[],'Red':[]}
    global CogCtrl
    global LingKnow
    global Stroop

    if CC == None:
        CogCtrl = CognitiveControl()
    else:
        CogCtrl = CC
        
    if LK == None:
        LingKnow = LinguisticKnowledge()
    else:
        LingKnow = LK
        
    if S == None:
        Stroop = StroopTaskRepresentation()
    else:
        Stroop = S
        

    LingKnow.input_activation(input_activation[0],input_activation[1],input_activation[2])
    Stroop.input_activation(input_activation[3],input_activation[4],input_activation[5])
    
    nodes_dict = {'ConfMon':CogCtrl.conflict_monitoring,'Biasing':CogCtrl.biasing,
                        'WK':LingKnow.world_knowledge,'MSK_ing':LingKnow.morphosyntactic_knowledge_ing,'MSK_ed':LingKnow.morphosyntactic_knowledge_ed,'SubjIsAgent':LingKnow.subj_is_agent,'SubjIsTheme':LingKnow.subj_is_theme,
                       'FontColor':Stroop.font_color,'Text_blue':Stroop.text_blue,'Text_red':Stroop.text_red,'Blue':Stroop.blue,'Red':Stroop.red}
    for key in activation_levels.keys():
        activation_levels[key].append(nodes_dict[key])

    while i <= max_iter:
        update_all(CogCtrl,LingKnow,Stroop)
        nodes_dict = {'ConfMon':CogCtrl.conflict_monitoring,'Biasing':CogCtrl.biasing,
                        'WK':LingKnow.world_knowledge,'MSK_ing':LingKnow.morphosyntactic_knowledge_ing,'MSK_ed':LingKnow.morphosyntactic_knowledge_ed,'SubjIsAgent':LingKnow.subj_is_agent,'SubjIsTheme':LingKnow.subj_is_theme,
                       'FontColor':Stroop.font_color,'Text_blue':Stroop.text_blue,'Text_red':Stroop.text_red,'Blue':Stroop.blue,'Red':Stroop.red}
        for key in activation_levels.keys():
            activation_levels[key].append(nodes_dict[key])
        i += 1
        MAX_ACT = max(LingKnow.subj_is_agent,LingKnow.subj_is_theme,Stroop.blue,Stroop.red)
        if MAX_ACT > activation_threshold:
            break
    
    # determine the final interpratation
    if [LingKnow.subj_is_agent,LingKnow.subj_is_theme,Stroop.blue,Stroop.red].count(MAX_ACT) > 1:
        winner = None
    if LingKnow.subj_is_agent == MAX_ACT:
        winner = 'SubjIsAgent'
    if LingKnow.subj_is_theme == MAX_ACT:
        winner = 'SubjIsTheme'
    if Stroop.blue == MAX_ACT:
        winner = 'Blue'
    if Stroop.red == MAX_ACT:
        winner = 'Red'
    
    return i, winner, CogCtrl, LingKnow, Stroop, activation_levels


def run_trial_sequence(trials:"list of tuples"):
    '''
    Trials is a list of tuples.
    Each tuple consists of 4 values that are the input activations of WK,MSK_ing,MSK_ed,text_blue,text_red,font_color (in that order) in a specific trial.
    '''
    CogCtrl = CognitiveControl()
    LingKnow = LinguisticKnowledge()
    Stroop = StroopTaskRepresentation()
    
    results = []
    for trial in trials:
        CogCtrl.between_trials_decay()
        LingKnow.between_trials_decay()
        Stroop.between_trials_decay()
        i, winner, CogCtrl, LingKnow, Stroop, Activations = run_trial(trial,CogCtrl,LingKnow,Stroop)
        results.append([i,winner,Activations])
        
    return results

# Figures

## Cognitive control makes conflict resolution faster

### In Stroop

In [4]:
user_biasing_mult = biasing_mult

# Congruent Stroop trial with biasing_mult = 0
del CogCtrl, LingKnow, Stroop
biasing_mult = 0
i, winner, CogCtrl, LingKnow, Stroop, Activations = run_trial(congruent_stroop)
RT_CONG_STROOP_NO_CC = i
print(i, winner)

# Congruent Stroop trial with biasing_mult as defined by user
del CogCtrl, LingKnow, Stroop
biasing_mult = user_biasing_mult
i, winner, CogCtrl, LingKnow, Stroop, Activations = run_trial(congruent_stroop)
RT_CONG_STROOP_WITH_CC = i
print(i, winner)

# Inongruent Stroop trial with biasing_mult = 0
del CogCtrl, LingKnow, Stroop
biasing_mult = 0
i, winner, CogCtrl, LingKnow, Stroop, Activations = run_trial(incongruent_stroop)
RT_INCONG_STROOP_NO_CC = i
print(i, winner)

# Incongruent Stroop trial with biasing_mult as defined by user
del CogCtrl, LingKnow, Stroop
biasing_mult = user_biasing_mult
i, winner, CogCtrl, LingKnow, Stroop, Activations = run_trial(incongruent_stroop)
RT_INCONG_STROOP_WITH_CC = i
print(i, winner)

# Figure
df = pd.DataFrame({'CC':['Without CC','With CC','Without CC','With CC'],
                    'Trial':['Congruent','Congruent','Incongruent','Incongruent'],
                    'RT':[RT_CONG_STROOP_NO_CC,RT_CONG_STROOP_WITH_CC,RT_INCONG_STROOP_NO_CC,RT_INCONG_STROOP_WITH_CC]})

fig = px.bar(df,x='Trial',y='RT',color='CC',barmode="group")
fig.show()


505 Blue
502 Blue
1032 Blue
667 Blue


The figure shows that cognitive control makes trials faster, particularly in incongruent trials.

### In Language

In [5]:
user_biasing_mult = biasing_mult

# Control sentence trial with biasing_mult = 0
del CogCtrl, LingKnow, Stroop
biasing_mult = 0
i, winner, CogCtrl, LingKnow, Stroop, Activations = run_trial(congruent_sentence)
RT_CONG_LANG_NO_CC = i
print(i, winner)

# control sentence trial with biasing_mult as defined by user
del CogCtrl, LingKnow, Stroop
biasing_mult = user_biasing_mult
i, winner, CogCtrl, LingKnow, Stroop, Activations = run_trial(congruent_sentence)
RT_CONG_LANG_WITH_CC = i
ActivationsCong = Activations
print(i, winner)

# Anomalous sentence trial with biasing_mult = 0
del CogCtrl, LingKnow, Stroop
biasing_mult = 0
i, winner, CogCtrl, LingKnow, Stroop, Activations = run_trial(anomalous_sentence)
RT_ANOM_LANG_NO_CC = i
print(i, winner)

# Anomalous sentence trial with biasing_mult as defined by user
del CogCtrl, LingKnow, Stroop
biasing_mult = user_biasing_mult
i, winner, CogCtrl, LingKnow, Stroop, Activations = run_trial(anomalous_sentence)
RT_ANOM_LANG_WITH_CC = i
ActivationsIncong = Activations
print(i, winner)

# Figure
df = pd.DataFrame({'CC':['Without CC','With CC','Without CC','With CC'],
                    'Trial':['Congruent','Congruent','Anomaly','Anomaly'],
                    'RT':[RT_CONG_LANG_NO_CC,RT_CONG_LANG_WITH_CC,RT_ANOM_LANG_NO_CC,RT_ANOM_LANG_WITH_CC]})

fig_WithWithout = px.bar(df,x='Trial',y='RT',color='CC',barmode="group")
fig_WithWithout.show()
fig_Cong_Bias = px.scatter(x=range(1,len(ActivationsCong['Biasing'])+1),y=ActivationsCong['Biasing'],title='Biasing')
fig_Cong_Agent = px.scatter(x=range(1,len(ActivationsCong['SubjIsAgent'])+1),y=ActivationsCong['SubjIsAgent'],title='SubjIsAgent')
fig_Cong_Theme = px.scatter(x=range(1,len(ActivationsCong['SubjIsTheme'])+1),y=ActivationsCong['SubjIsTheme'],title='SubjIsTheme')
fig_Cong_Bias.show()
fig_Cong_Agent.show()
fig_Cong_Theme.show()
fig_Incong_Bias = px.scatter(x=range(1,len(ActivationsIncong['Biasing'])+1),y=ActivationsIncong['Biasing'])
fig_Incong_Agent = px.scatter(x=range(1,len(ActivationsIncong['SubjIsAgent'])+1),y=ActivationsIncong['SubjIsAgent'])
fig_Incong_Theme = px.scatter(x=range(1,len(ActivationsIncong['SubjIsTheme'])+1),y=ActivationsIncong['SubjIsTheme'])
fig_Incong_Bias.show()
fig_Incong_Agent.show()
fig_Incong_Theme.show()


505 SubjIsTheme
502 SubjIsTheme
1032 SubjIsTheme
667 SubjIsTheme


The figure shows that the same cognitive control system also makes linguistic trials faster, particularly in the anomalous trials.

## Cross-Task Adaptation

In [6]:
all_results = {}

# A Congruent Stroop -> Control Sentence sequence
del CogCtrl, LingKnow, Stroop
results = run_trial_sequence((congruent_stroop,congruent_sentence))
RT_CongCong = results[1][0]
print([(r[0],r[1]) for r in results])
all_results['CongCong'] = results

# An Inongruent Stroop -> Control Sentence sequence
del CogCtrl, LingKnow, Stroop
results = run_trial_sequence((incongruent_stroop,congruent_sentence))
RT_IncongCong = results[1][0]
print([(r[0],r[1]) for r in results])
all_results['IncongCong'] = results

# A Congruent Stroop -> Anomalous Sentence sequence
del CogCtrl, LingKnow, Stroop
results = run_trial_sequence((congruent_stroop,anomalous_sentence))
RT_CongAnom = results[1][0]
print([(r[0],r[1]) for r in results])
all_results['CongAnom'] = results

# An Incongruent Stroop -> Anomalous Sentence sequence
del CogCtrl, LingKnow, Stroop
results = run_trial_sequence((incongruent_stroop,anomalous_sentence))
RT_IncongAnom = results[1][0]
print([(r[0],r[1]) for r in results])
all_results['IncongAnom'] = results

# Figure
df = pd.DataFrame({'PrevStroop':['Congruent','Incongruent','Congruent','Incongruent'],
                    'Trial':['Congruent','Congruent','Anomaly','Anomaly'],
                    'RT':[RT_CongCong,RT_IncongCong,RT_CongAnom,RT_IncongAnom]})

fig = px.bar(df,x='PrevStroop',y='RT',color='Trial',barmode="group")
fig.show()


[(502, 'Blue'), (501, 'SubjIsTheme')]
[(667, 'Blue'), (416, 'SubjIsTheme')]
[(502, 'Blue'), (663, 'SubjIsTheme')]
[(667, 'Blue'), (545, 'SubjIsTheme')]


The figure shows that the engagment of cognitive control in an Incongruent Stroop Trial makes the subsequent Anomalous Sentence trial faster (and it affects Control Sentences less).